In [1]:
import tensorflow as tf
from object_detection.utils import dataset_util
import os
import numpy as np

2023-10-22 14:37:54.312708: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-22 14:37:54.351025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 14:37:55.147593: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def convert_label(label, shape=(500, 500)):
    # Parse the label data from the txt file
    _, x, y, w, h = label.strip().split()
    x, y, w, h = float(x)*shape[0], float(y)*shape[1], float(w)*shape[0], float(h)*shape[1]
    # Convert the label data to the format expected by the tf.record file
    xmin = x - w/2
    ymin = y - h/2
    xmax = x + w/2
    ymax = y + h/2
    label = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
    return label

In [12]:
path = '../datasets/version1/test/'
output_file = os.path.join(path, 'dataset.tfrecord')

In [13]:
images_path = os.path.join(path, 'images')
labels_path = os.path.join(path, 'labels')

In [14]:
images_filepaths = os.listdir(images_path)
labels_filepaths = os.listdir(labels_path)

images_filepaths.sort()
labels_filepaths.sort()


In [15]:
with tf.io.TFRecordWriter(output_file) as writer:
    
    for label_filepath, image_filepath in zip(labels_filepaths, images_filepaths):

        with open(os.path.join(labels_path, label_filepath), 'r') as f:
            label_data = f.read()

        with tf.io.gfile.GFile(os.path.join(images_path, image_filepath), 'rb') as f:
            image_data = f.read()

        xmin, ymin, xmax, ymax = convert_label(label_data)
        
        example = tf.train.Example(features=tf.train.Features(feature={
            'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data])),
            'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[500])),#dataset_util.int64_feature(500),
            'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[500])),#dataset_util.int64_feature(500),
            'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'png'])),
            'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=[xmin])),
            'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=[ymin])),
            'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=[xmax])),
            'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=[ymax])),
            'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'usb_cable'])),
            'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=[0])),
            
        }))

        writer.write(example.SerializeToString())